# IMPORT LIBRARIES AND LOAD DATA

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt

# Define parameters
data_dir = '/kaggle/input/food-image-classification-dataset/Food Classification dataset'  # Path to the directory with images
nutrition_df = pd.read_csv('/kaggle/input/nutrition-info-csv/nutrition-info')
batch_size = 32
img_height = 224  # Standard input size for MobileNetV2
img_width = 224
validation_split = 0.2  # 20% of data used for validation

# Data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=validation_split,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create train and validation generators
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'
)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2


# CNN MODEL WITH MOBILENET

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation="softmax")
])

print(model.summary())



In [ ]:
# Display class names
class_names = list(train_generator.class_indices.keys())
print("Class names:", class_names)

# Load the MobileNetV2 model pre-trained on ImageNet without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the base model layers to retain pre-trained features

# Build the model on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')  # Adjust for the number of classes
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Display class names
class_names = list(train_generator.class_indices.keys())
print("Class names:", class_names)

# Display a few sample images from the dataset with labels
plt.figure(figsize=(10, 10))
for images, labels in train_generator:
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(class_names[int(labels[i])])
        plt.axis("off")
    break

# Train the model
epochs = 1  # Increase epochs for potentially better performance
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_generator, verbose=2)
print("\nValidation accuracy:", val_acc)


In [ ]:
train_loss, train_acc = model.evaluate(train_generator, verbose =2)
val_loss, val_acc = model.evaluate(validation_generator,verbose =2)

print(f"\nFinal Training Accuracy: {train_acc:.2f}")
print(f"Final Training Loss: {train_loss:.2f}")
print(f"Final Validation Accuracy: {val_acc:.2f}")
print(f"Final Validation Loss: {val_loss:.2f}")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

validation_generator.reset()  # Reset the generator for new predictions
predictions = model.predict(validation_generator, steps=validation_generator.samples // batch_size)
predicted_classes = np.argmax(predictions, axis=1)  # Convert probabilities to class labels

# Get true classes
true_classes = validation_generator.classes

# Create confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Function to predict food item and nutritional values
def predict_nutrition(image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    img = np.expand_dims(img, axis=0) / 255.0  # Normalize the image

    # Make predictions
    predictions = model.predict(img)
    predicted_label_index = np.argmax(predictions[0])
    predicted_food_item = label_encoder.inverse_transform([predicted_label_index])[0]

    # Get nutritional information
    nutritional_info = nutrition_df[nutrition_df['Food Label'] == predicted_food_item]
    
    if not nutritional_info.empty:
        return predicted_food_item, nutritional_info.to_dict('records')[0]
    else:
        return predicted_food_item, None

# Example usage
sample_image_path = 'path_to_sample_image.jpg'  # Update with the path to your sample image
predicted_food, nutrition = predict_nutrition(sample_image_path)

# Output the results
if nutrition is not None:
    print(f"Predicted Food Item: {predicted_food}")
    print(f"Nutritional Information: {nutrition}")
else:
    print("Nutritional information not found.")